# Generalization and Model Validation

In [ ]:
import numpy as np
import pods
data = pods.datasets.olympic_marathon_men()
x = data['X']
y = data['Y']

%matplotlib inline
import pylab as plt

### Question 1: Sample Code

In [ ]:
#### Question 1 Answer Code
# Write code for you answer to this question in this box
# Do not delete these comments, otherwise you will get zero for this answer.
# Make sure your code has run and the answer is correct *before* submitting your notebook for marking.

def linear(x):
    return np.hstack([x, np.ones((len(x), 1))])

def prediction(w, x, basis):
    Phi = basis(x)
    f = np.dot(Phi, w)
    return f

def objective(w, x, y, basis):
    Phi = basis(x)
    f = np.dot(Phi, w)
    diff = y - f
    e = np.dot((y - f).T, (y - f))
    return  e

def fit(x, y, basis):
    Phi = basis(x)
    w = np.linalg.solve(np.dot(Phi.T, Phi), np.dot(Phi.T,y))
    return w

# linear fit with marathon data
w = fit(x, y, linear)
f = prediction(w, x, linear)
e = objective(w, x, y, linear)

plt.plot(x, y, 'rx')
plt.plot(x, f, 'b-')
plt.axis([1890, 2020, 2.5, 5.5])
plt.title("error: %f" %(e))

### Question 2: Sample Code

In [ ]:
#### Question 2 Answer Code
# Write code for you answer to this question in this box
# Do not delete these comments, otherwise you will get zero for this answer.
# Make sure your code has run and the answer is correct *before* submitting your notebook for marking.

def quadratic(x):
    Phi = np.hstack([np.ones((len(x), 1)), x, x**2])
    return Phi

# quadratic fit with marathon data
w = fit(x, y, quadratic)
f = prediction(w, x, quadratic)
e = objective(w, x, y, quadratic)

plt.plot(x, y, 'rx')
plt.plot(x, f, 'b-')
plt.axis([1890, 2020, 2.5, 5.5])
plt.title("error: %f" %(e))

### Question 3: Sample Code

In [ ]:
#### Question 3 Answer Code
# Write code for you answer to this question in this box
# Do not delete these comments, otherwise you will get zero for this answer.
# Make sure your code has run and the answer is correct *before* submitting your notebook for marking.

# select indices of data to 'hold out'
indices_hold_out = np.flatnonzero(x>1980)

# training set
x_train = np.delete(x, indices_hold_out, axis=0)
y_train = np.delete(y, indices_hold_out, axis=0)

# hold out set
x_valid = np.take(x, indices_hold_out, axis=0)
y_valid = np.take(y, indices_hold_out, axis=0)

# linear fit
w_train_linear = fit(x_train, y_train, linear)
f_train_linear = prediction(w_train_linear, x, linear)
e_valid_linear = objective(w_train_linear, x_valid, y_valid, linear)

# quadratic fit
w_train_quadratic = fit(x_train, y_train, quadratic)
f_train_quadratic = prediction(w_train_quadratic, x, quadratic)
e_valid_quadratic = objective(w_train_quadratic, x_valid, y_valid, quadratic)

# print & plot
print('e_valid_linear', e_valid_linear, 'e_valid_quadratic', e_valid_quadratic,'quadratic function perform better')

plt.plot(x_train, y_train, 'rx')
plt.plot(x_valid, y_valid, 'gx')
plt.plot(x, f_train_linear, 'y-')
plt.plot(x, f_train_quadratic, 'b-')
plt.axis([1890, 2020, 2.5, 5.5])
plt.title('e_valid_linear %f  e_valid_quadratic %f'%(e_valid_linear, e_valid_quadratic))

### Question 4 Sample Code

In [ ]:
#### Question 4 Answer Code
# Write code for you answer to this question in this box
# Do not delete these comments, otherwise you will get zero for this answer.
# Make sure your code has run and the answer is correct *before* submitting your notebook for marking.

def polynomial(x, degree, loc, scale):
    degrees = np.arange(degree+1)
    return ((x-loc)/scale)**degrees

def prediction(w, x, basis={'function':polynomial, 'degree':1, 'loc':0., 'scale':10.}):
    Phi = basis['function'](x, basis['degree'], basis['loc'], basis['scale'])    
    return np.dot(Phi,w)

def objective(w, x, y, basis={'function':polynomial, 'degree':1, 'loc':0., 'scale':10.}):
    Phi = basis['function'](x, basis['degree'], basis['loc'], basis['scale'])
    return ((y - np.dot(Phi,w))**2).sum()

def fit(x, y, basis={'function':polynomial, 'degree':1, 'loc':0., 'scale':10.}):
    Phi = basis['function'](x, basis['degree'], basis['loc'], basis['scale'])
    w = np.linalg.solve(np.dot(Phi.T, Phi), np.dot(Phi.T, y))
    return w

# polynomial fit
degree_range = [0, 17]
degrees = np.arange(degree_range[0], degree_range[1] + 1)
train_err = np.zeros(degree_range[1] - degree_range[0] + 1)
hold_err = np.zeros(degree_range[1] - degree_range[0] + 1)
for degree in degrees:
    basis = {'function':polynomial, 'degree':degree, 'loc':1956., 'scale':120.}
    w = fit(x_train, y_train, basis)
    train_err[degree] = objective(w, x_train, y_train, basis)
    hold_err[degree] = objective(w, x_valid, y_valid, basis)

# print & plot
plt.figure()
plt.plot(degrees, train_err, 'r*')
plt.ylabel('Training Error')
plt.xlabel('Max degree of polynomial')

plt.figure()
plt.semilogy(degrees, hold_err, 'b*')
plt.ylabel('Validation Error (log scale)')
plt.xlabel('Max degree of polynomial')

print('Degree of polynomial with minimum training error : ', train_err.argmin())
print('Degree of polynomial with minimum validation error : ', hold_err.argmin())

### Question 5: Sample Code

In [ ]:
#### Question 5 Answer Code
# Write code for you answer to this question in this box
# Do not delete these comments, otherwise you will get zero for this answer.
# Make sure your code has run and the answer is correct *before* submitting your notebook for marking.

degree_range = [0, 17]
degrees = np.arange(degree_range[0], degree_range[1] + 1)
train_err = np.zeros(degree_range[1] - degree_range[0] + 1)
hold_err = np.zeros(degree_range[1] - degree_range[0] + 1)

for degree in degrees:
    basis = {'function':polynomial, 'degree':degree, 'loc':1956., 'scale':120.}
    train_err_local = np.zeros(x.shape[0])
    hold_err_local = np.zeros(x.shape[0])
    
    for idx in range(0, x.shape[0]):
        # training set
        x_train = np.delete(x, idx, axis=0)
        y_train = np.delete(y, idx, axis=0)

        # hold out set
        x_valid = x[idx]
        y_valid = y[idx]
        
        w = fit(x_train, y_train, basis)
        x_pred = np.linspace(1890, 2020, 200)[:,None]
        y_pred = prediction(w, x_pred, basis)
        train_err_local[idx] = objective(w, x_train, y_train, basis)
        hold_err_local[idx] = objective(w, x_valid, y_valid, basis)
    
    train_err[degree] = train_err_local.mean()
    hold_err[degree] = hold_err_local.mean()

# print and plot
plt.figure()
plt.plot(degrees, train_err, 'r*')
plt.ylabel('Training Error')
plt.xlabel('Max degree of polynomial')

plt.figure()
plt.semilogy(degrees, hold_err, 'b*')
plt.ylabel('Validation Error (log scale)')
plt.xlabel('Max degree of polynomial')

print('Degree of polynomial with minimum training error : ', train_err.argmin())
print('Degree of polynomial with minimum validation error : ', hold_err.argmin())

### Question 6: Sample Code

In [ ]:
#### Question 6 Answer Code
# Write code for you answer to this question in this box
# Do not delete these comments, otherwise you will get zero for this answer.
# Make sure your code has run and the answer is correct *before* submitting your notebook for marking.

# 5-fold cross validation
k = 5
fold_size = int(x.shape[0]/k)
num_extra_pts = x.shape[0]%k

folds = []
idx_rand = np.random.permutation(np.arange(x.shape[0]))

for idx in range(k):
    if num_extra_pts > 0:
        folds += [idx_rand[idx*fold_size : (idx+1)*fold_size + 1]]
        num_extra_pts -= 1
    else:
        folds += [idx_rand[idx*fold_size : (idx+1)*fold_size]]

degree_range = [0, 17]
degrees = np.arange(degree_range[0], degree_range[1] + 1)
train_err = np.zeros(degree_range[1] - degree_range[0] + 1)
hold_err = np.zeros(degree_range[1] - degree_range[0] + 1)

for degree in degrees:
    basis = {'function':polynomial, 'degree':degree, 'loc':1956., 'scale':120.}
    train_err_local = np.zeros(x.shape[0])
    hold_err_local = np.zeros(x.shape[0])
    
    for idx in range(k):
        # training set
        x_train = np.delete(x, folds[idx], axis=0)
        y_train = np.delete(y, folds[idx], axis=0)

        # hold out set
        x_valid = np.take(x, folds[idx], axis=0)
        y_valid = np.take(y, folds[idx], axis=0)
        
        w = fit(x_train, y_train, basis)
        y_pred = prediction(w, x_pred, basis)
        train_err_local[idx] = objective(w, x_train, y_train, basis)
        hold_err_local[idx] = objective(w, x_valid, y_valid, basis)
    
    train_err[degree] = train_err_local.mean()
    hold_err[degree] = hold_err_local.mean()

# print & plot
plt.figure()
plt.plot(degrees, train_err, 'r*')
plt.ylabel('Training Error')
plt.xlabel('Max degree of polynomial')

plt.figure()
plt.semilogy(degrees, hold_err, 'b*')
plt.ylabel('Validation Error (log scale)')
plt.xlabel('Max degree of polynomial')

print('Degree of polynomial with minimum training error : ', train_err.argmin())
print('Degree of polynomial with minimum validation error : ', hold_err.argmin())

print('The 5-fold cross validation DOES NOT always select the same model')